# Setup

Locally store your Planet API key and start a session. Create a funciton to print json objects.

In [55]:
import os
import json
import requests
PLANET_API_KEY = os.getenv('PL_API_KEY')

# Setup Planet Data API base URL
URL = "https://api.planet.com/data/v1"

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (PLANET_API_KEY, "")

res = session.get(URL)
res.status_code


# Helper function to print formatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))


# Stats

Here you will perform a statistics search of planets database, while getting familiar with the various filtering options. 

In [56]:
# Setup the stats URL
stats_url = "{}/stats".format(URL)

Set up the date filter to a time of your choice.

In [102]:
date_filter = {
    "type": "DateRangeFilter", # Type of filter -> Date Range
    "field_name": "acquired", # The field to filter on: "acquired" -> Date on which the "image was taken"
    "config": {
        "gte": "2000-01-01T00:00:00.000Z", # "gte" -> Greater than or equal to
    }
}

Pick a coordinate over your AOI. You can select it from [google maps.](https://www.google.com/maps/)

In [103]:
geometry = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": {
    "type": "Point",
    "coordinates": [
        0,
        0
    ]
  }
}

Set up a cloud filter. Remember the `lt` stands for "less than" and `gt` stands for "greater than".

In [104]:
# Setup Cloud Filter
cloud_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lt": 0.0,
    "gt": 0.0
  }
}


Now join all of the filters together.

In [105]:
and_filter = {
    "type": "AndFilter",
    "config": [geometry, date_filter, cloud_filter]
}

p(and_filter)

{
  "type": "AndFilter",
  "config": [
    {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": {
        "type": "Point",
        "coordinates": [
          -104,
          39
        ]
      }
    },
    {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": "2022-01-20T00:00:00.000Z"
      }
    },
    {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lt": 0.3,
        "gt": 0.0
      }
    }
  ]
}


Search the database to see how many results fall in this category. Insert the [Item Type](https://developers.planet.com/docs/apis/data/items-assets/) of your choice.

In [106]:
item_types = ["PSScene4Band"]

# Setup the request
request = {
    "item_types" : item_types,
    "interval" : "year",
    "filter" : and_filter
}

# Send the POST request to the API stats endpoint
res=session.post(stats_url, json=request)

# Print response
p(res.json())



{
  "buckets": [
    {
      "count": 1,
      "start_time": "2022-01-01T00:00:00.000000Z"
    }
  ],
  "interval": "year",
  "utc_offset": "+0h"
}


Congratulations, you have performed your fist statistics search using the Data API!

# Quick Search

Here you will perform a search for specific image ID's, using the search criterias you defined above, in order to Download them!

All the code you need is below in small chuncks, however they are in the wrong order! Re-order them correctly to Download your image.

In [124]:
#Send a request to the item's asset url in order to activate it for download
#This step might take some time
asset_activated = False
while asset_activated == False:
    res = session.get(assets_url)
    assets = res.json()
    asset_status = image["status"]
    if asset_status == 'active':
        asset_activated = True
        print("Asset is active and ready to download")    
p(image)


# Get the links for the item and find out what asset types are available
assets_url = feature["_links"]["assets"]
res = session.get(assets_url)
assets = res.json()
print(assets.keys())


# Setup the quick search endpoint url
# Create a request
quick_url = "{}/quick-search".format(URL)
item_types = ["PSScene4Band"]
request = {
    "item_types" : item_types,
    "filter" : and_filter
}


# Print the assets location endpoint for download
# Clicking on this url will download the image
location_url = image["location"]
print(location_url)


# Send the POST request to the API quick search endpoint
# Select the first feature from the search results and print its ID
# print the result
res = session.post(quick_url, json=request)
geojson = res.json()
feature = geojson["features"][0]
p(feature["id"])


# Pick an asset type
# Send a request to the activation url to activate the item
image = assets["analytic"]
activation_url = image["_links"]["activate"]
res = session.get(activation_url)
p(res.status_code)

"20220126_173923_29_2402"
dict_keys(['analytic', 'analytic_dn', 'analytic_dn_xml', 'analytic_sr', 'analytic_xml', 'basic_analytic', 'basic_analytic_dn', 'basic_analytic_dn_nitf', 'basic_analytic_dn_rpc', 'basic_analytic_dn_rpc_nitf', 'basic_analytic_dn_xml', 'basic_analytic_dn_xml_nitf', 'basic_analytic_nitf', 'basic_analytic_rpc', 'basic_analytic_rpc_nitf', 'basic_analytic_xml', 'basic_analytic_xml_nitf', 'basic_udm', 'basic_udm2', 'udm', 'udm2'])
204
Asset is active and ready to download
{
  "_links": {
    "_self": "https://api.planet.com/data/v1/assets/eyJpIjogIjIwMjIwMTI2XzE3MzkyM18yOV8yNDAyIiwgImMiOiAiUFNTY2VuZTRCYW5kIiwgInQiOiAiYW5hbHl0aWMiLCAiY3QiOiAiaXRlbS10eXBlIn0",
    "activate": "https://api.planet.com/data/v1/assets/eyJpIjogIjIwMjIwMTI2XzE3MzkyM18yOV8yNDAyIiwgImMiOiAiUFNTY2VuZTRCYW5kIiwgInQiOiAiYW5hbHl0aWMiLCAiY3QiOiAiaXRlbS10eXBlIn0/activate",
    "type": "https://api.planet.com/data/v1/asset-types/analytic"
  },
  "_permissions": [
    "download"
  ],
  "expires_at": "2

Use the same request that you had created before in order to search for the imagery.